In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display
%matplotlib inline

# Chapter 4. Representing Data and Engineering Features

So far, we’ve assumed that our data comes in as a two-dimensional array of floating-point numbers, where each column is a [continuous feature](https://www.mathsisfun.com/data/data-discrete-continuous.html) that describes the data points.  
For many applications, this is not how the data is collected.  
A particularly common type of feature is the [categorical features](http://www.dummies.com/education/math/statistics/types-of-statistical-data-numerical-categorical-and-ordinal/).  
Also known as [discrete features](https://www.mathsisfun.com/data/data-discrete-continuous.html), these are usually not numeric.  
The distinction between categorical features and continuous features is analogous to the distinction between classification and regression, only on the input side rather than the output side.  
Examples of continuous features that we have seen are pixel brightnesses and size measurements of plant flowers.  
Examples of categorical features are the brand of a product, the color of a product, or the department (books, clothing, hardware) it is sold in.  
These are all properties that can describe a product, but they don’t vary in a continuous way.  
A product belongs either in the clothing department or in the books department.  
There is no middle ground between books and clothing, and no natural order for the different categories (books is not greater or less than clothing, hardware is not between books and clothing, etc.).

Regardless of the types of features your data consists of, how you represent them can have an enormous effect on the performance of machine learning models.  
We saw in Chapters 2 and 3 that scaling of the data is important.  
In other words, if you don’t rescale your data (say, to unit variance), then it makes a difference whether you represent a measurement in centimeters or inches.  
We also saw in Chapter 2 that it can be helpful to *augment* your data with additional features, like adding interactions (products) of features or more general polynomials.

The question of how to represent your data best for a particular application is known as feature engineering, and it is one of the main tasks of data scientists and machine learning practitioners trying to solve real-world problems.  
Representing your data in the right way can have a bigger influence on the performance of a supervised model than the exact parameters you choose.  
In this chapter, we will first go over the important and very common case of categorical features, and then give some examples of helpful transformations for specific combinations of features and models.

## Categorical Variables

As an example, we will use the dataset of adult incomes in the United States, derived from the 1994 census database.  
The task of the adult dataset is to predict whether a worker has an income of over \$50,000 or under \$50,000.  
The features in this dataset include the workers’ ages, how they are employed (self employed, private industry employee, government employee, etc.), their education, their gender, their working hours per week, occupation, and more.  
Table 4-1 shows the first few entries in the dataset:

![title](images/adult_incomes_data.png)

The task is phrased as a classification task with the two classes being income `<=50k` and `>50k`.  
It would also be possible to predict the exact income, and make this a regression task.  
However, that would be much more difficult, and the 50K division is interesting to understand on its own.  
In this dataset, `age` and `hours-per-week` are continuous features, which we know how to treat.  
The `workclass`, `education`, `sex`, and `occupation` features are categorical, however.  
All of them come from a fixed list of possible values, as opposed to a range, and denote a qualitative property, as opposed to a quantity.  
As a starting point, let’s say we want to learn a logistic regression classifier on this data.  
We know from Chapter 2 that a logistic regression makes predictions, $ŷ$, using the following formula:

$ŷ = w[0] * x[0] +w[1] * x[1] + \ldots + w[p] * x[p] + b > 0$

where $w[i]$ and $b$ are coefficients learned from the training set and $x[i]$ are the input features.  
This formula makes sense when $x[i]$ are numbers, but not when $x[2]$ is `"Masters"` or `"Bachelors"`.  
Clearly we need to represent our data in some different way when applying logistic regression.  
The next section will explain how we can overcome this problem.

### One-Hot-Encoding (Dummy Variables)

By far the most common way to represent categorical variables is using the [one-hot-encoding](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science) or *one-out-of-N encoding*, also known as [dummy variables](https://en.wikipedia.org/wiki/Dummy_variable_%28statistics%29).  
The idea behind dummy variables is to replace a categorical variable with one or more new features that can have the values 0 and 1.  
The values 0 and 1 make sense in the formula for linear binary classification (and for all other models in `scikit-learn`), and we can represent any number of categories by introducing one new feature per category, as described here.  
Let’s say for the workclass feature we have possible values of `"Government Employee"`, `"Private Employee"`, `"Self Employed"`, and `"Self Employed Incorporated"`.  
To encode these four possible values, we create four new features, called `"Government Employee"`, `"Private Employee"`, `"Self Employed"`, and `"Self Employed Incorporated"`.  
A feature is 1 if `workclass` for this person has the corresponding value and 0 otherwise, so exactly one of the four new features will be 1 for each data point.  
This is why this is called *one-hot* or *one-out-of-N* encoding.

The principle is illustrated in Table 4-2.  
A single feature is encoded using four new features.  
When using this data in a machine learning algorithm, we would drop the original `workclass` feature and only keep the 0-1 features.

![title](images/Table_4-2.png)

**NOTE**  
The one-hot encoding we use is quite similar, but not identical, to the dummy coding used in statistics.  
For simplicity, we encode each category with a different binary feature.  
In statistics, it is common to encode a categorical feature with k different possible values into k–1 features (the last one is represented as all zeros).  
This is done to simplify the analysis (more technically, this will avoid making the data matrix rank-deficient.

There are two ways to convert your data to a one-hot encoding of categorical variables, using either [pandas](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) or [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).  
At the time of writing, using pandas is slightly easier, so let’s go this route.  
First we load the data using pandas from a comma-separated values (CSV) file:

In [24]:
import os
# The file has no headers naking the columns, so we pass 
# header=None and provide the column names explicitly in "names".
adult_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")
print(adult_path)

/Users/benjamingrove/.pyenv/versions/3.6.1/lib/python3.6/site-packages/mglearn/data/adult.data


In [25]:
data = pd.read_csv(adult_path, header=None, index_col=False,
       names=['age', 'workclass', 'fnlwgt', 'education',
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'gender', 'capital-gain',
              'capital-loss', 'hours-per-week', 'native-country',
              'income'])
# For illustrative purposes, we'll only select some of the columns:
data = data[['age', 'workclass', 'education', 'gender',
             'hours-per-week', 'occupation', 'income']]
# IPython.display allows nice output formatting within the 
# Jupyter notebook:
display(data.head())

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


#### Checking string-encoded categorical data

After reading a dataset like this, it is often good to first check if a column actually contains meaningful categorical data.  
When working with data that was input by humans (say, users on a website), there might not be a fixed set of categories, and differences in spelling and capitalization might require preprocessing.  
For example, it might be that some people specified gender as “male” and some as “man,” and we might want to represent these two inputs using the same category.  
A good way to check the contents of a column is using the `value_counts` method of a `pandas` `Series` (the type of a single column in a `DataFrame`), to show us what the unique values are and how often they appear:

In [26]:
print(data.gender.value_counts())

 Male      21790
 Female    10771
Name: gender, dtype: int64


We can see that there are exactly two values for gender in this dataset, `Male` and `Female`, meaning the data is already in a good format to be represented using one-hot-encoding.  
In a real application, you should look at all columns and check their values.  
We will skip this here for brevity’s sake.  
There is a very simple way to encode the data in pandas, using the `get_dummies()` function.  
The [get_dummies()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) function automatically transforms all columns that have object type (like strings) or are categorical (which is a special `pandas` concept that we haven’t talked about yet):

In [27]:
print("Original Features: \n", list(data.columns))

Original Features: 
 ['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']


In [28]:
data_dummies = pd.get_dummies(data)
print("Features After Applying get_dummies(): \n",
      list(data_dummies.columns))

Features After Applying get_dummies(): 
 ['age', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'gender_ Female', 'gender_ Male', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct', 'occupation_ Other-service', 'occupation_ Priv-house-serv', 'occupation_ Prof-specialty'

You can see that the continuous features `age` and `hours-per-week` were not touched, while the categorical features were expanded into one new feature for each possible value:

In [29]:
data_dummies.head()

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,39,40,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,50,13,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,38,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,53,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,28,40,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


We can now use the values attribute to convert the `data_dummies` `DataFrame` into a `NumPy` array, and then train a machine learning model on it.  
Be careful to separate the target variable (which is now encoded in two income columns) from the data before training a model.  
Including the output variable, or some derived property of the output variable, into the feature representation is a very common mistake in building supervised machine learning models.

**WARNING**  
Be careful: column indexing in pandas includes the end of the range, so `'age'`:`'occupation_ Transport-moving'` is inclusive of `occupation_` `Transport-moving`.  
This is different from slicing a NumPy array, where the end of a range is not included: for example, `np.arange(11)[0:10]` doesn’t include the entry with index 10.

In this case, we extract only the columns containing features -- that is, all columns from `age` to `occupation_ Transport-moving`.  
This range contains all of the features but not the target:

In [30]:
features = data_dummies.loc[:, 'age':'occupation_ Transport-moving']
# Extract NumPy arrays:
X = features.values
print("X[:5]: \n", X[:5], "\n")
y = data_dummies['income_ >50K'].values
print("y[:5]: \n", y[:5], "\n")
print("X.shape: {}  y.shape: {}".format(X.shape, y.shape))

X[:5]: 
 [[39 40  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0
   0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [50 13  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
   0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [38 40  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
   0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [53 40  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [28 40  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
   0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]] 

y[:5]: 
 [0 0 0 0 0] 

X.shape: (32561, 44)  y.shape: (32561,)


Now the data is represented in a way that `scikit-learn` can work with, and we can proceed as usual:

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print("Test Score: {:.2f}".format(logreg.score(X_test, y_test)))

Test Score: 0.81


**WARNING**  
In this example, we called `get_dummies()` on a `DataFrame` containing both the training and the test data.  
This is important to ensure categorical values are represented in the same way in the training set and the test set.  
Imagine we have the training and test sets in two different `DataFrames`.  
If the `"Private Employee"` value for the `workclass` feature does not appear in the test set, `pandas` will assume there are only three possible values for this feature and will create only three new dummy features.  
Now our training and test sets have different numbers of features, and we can’t apply the model we learned on the training set to the test set anymore.  
Even worse, imagine the workclass feature has the values `"Government Employee"` and `"Private Employee"` in the training set, and `"Self Employed"` and `"Self Employed Incorporated"` in the test set.  
In both cases, `pandas` will create two new dummy features, so the encoded `DataFrames` will have the same number of features.  
However, the two dummy features have entirely different meanings in the training and test sets.  
The column that means `"Government Employee"` for the training set would encode `"Self Employed"` for the test set.  
If we built a machine learning model on this data it would work very badly, because it would assume the columns mean the same things (because they are in the same position) when in fact they mean very different things.  
To fix this, either call `get_dummies()` on a DataFrame that contains both the training and the test data points, or make sure that the column names are the same for the training and test sets after calling `get_dummies()`, to ensure they have the same semantics.

### Numbers Can Encode Categoricals